In [2]:
import numpy as np
from pandas import DataFrame, Series
import pandas as pd

In [3]:
# !pip install selenium

In [4]:
from selenium import webdriver

In [5]:
import time

In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
from selenium.common.exceptions import NoSuchElementException

from selenium.common.exceptions import TimeoutException

In [8]:
df= pd.read_csv('./watcha_user/user_id_sort_by_rating_count_final.csv', index_col = 0)

df

,id,rating_count
0,17ovVgGBL5zyn,20120.0
1,YMKqmjXMkvloD,16657.0
2,Qgy51zEAZxDjk,11152.0
3,XZBm5RYOavd46,10716.0
4,WRQxDOw1V5dl9,10676.0
...,...,...
215443,nb4xk8PZKqOAz,0.0
215444,zNM5NpDpdq26j,0.0
215445,nkPvrPm9Krxga,0.0
215446,87Gv7jyBGqE6o,0.0


In [9]:
df = df.loc[df.rating_count>5]
df.reset_index(drop=True, inplace=True)

In [10]:
df

,id,rating_count
0,17ovVgGBL5zyn,20120.0
1,YMKqmjXMkvloD,16657.0
2,Qgy51zEAZxDjk,11152.0
3,XZBm5RYOavd46,10716.0
4,WRQxDOw1V5dl9,10676.0
...,...,...
207788,zNM5NbVn2526j,6.0
207789,87Gv7GXjJvE6o,6.0
207790,WRQxDPYZnxdl9,6.0
207791,3BnvwnbbDKqPA,6.0


In [11]:
from tqdm import tqdm_notebook

In [12]:
from queue import Queue

from multiprocessing.pool import ThreadPool

In [13]:
def login(driver):
    """login 후 login 완료된 driver return"""
    driver.get('https://watcha.com/ko-KR')

    xpath_login = """//*[@id="root"]/div/div[1]/header/nav/div/div/ul/li[2]/button"""
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath_login))
    )
    driver.find_element_by_xpath(xpath_login).click()

    email_ = "hunji11@naver.com"
    pw_ = 'teamproject10'
#     email_ = "sistersbomb@naver.com"
#     pw_ = "qorzjtm1346"
    elem_id = driver.find_element_by_id('sign_in_email')
    elem_id.clear()
    elem_id.send_keys(email_)

    elem_pw = driver.find_element_by_id('sign_in_password')
    elem_pw.clear()
    elem_pw.send_keys(pw_)

    xpath_login_click = """//*[@id="root"]/div/div[2]/div/div/div/section/div/div/form/button"""
    driver.find_element_by_xpath(xpath_login_click).click()
    return driver

In [14]:
import requests
from bs4 import BeautifulSoup

In [15]:
def user_info(driver, test_df):
    try:
        lst_true = []
        lst_false = []
        for user_id in tqdm_notebook(test_df['id']):
            temp_ = []
            url = f'https://watcha.com/ko-KR/users/{user_id}/contents/movies/ratings'
            if int(test_df.loc[test_df.id==user_id, 'rating_count'])<10:
                res = requests.get(url)
                html = res.text
                soup = BeautifulSoup(html, 'html.parser')
                soups = soup.find_all(class_='e3fgkal0')
                for elem in soups:
                    mv_code=elem.find('a').get('href').split('/')[-1]
                    ratings=float(elem.find(class_='e3fgkal4').text.lstrip('★'))
                    temp_.append((user_id, mv_code, ratings))
                    lst_true.append((user_id, mv_code, ratings))
            else:
                driver.get(url)
                last_height = str(driver.execute_script('return document.documentElement.scrollHeight'))
                repeat_num=1
                while True:
                    driver.execute_script(f'window.scrollTo(0, {last_height});')
                    time.sleep(0.5)
                    try:
                        WebDriverWait(driver, 5).until(
                            lambda x: str(x.execute_script('return document.documentElement.scrollHeight')) != last_height
                        )
                    except TimeoutException:
                        difference = len(driver.find_elements_by_class_name('e3fgkal0'))\
                              - int(test_df.loc[test_df.id==user_id, 'rating_count'])
                        if difference<-9 and repeat_num<21:
                            print('=============================================================')
                            print(user_id, f": 개수가 맞지않아 {repeat_num}번째 재실행 중(오차 {difference}개)")
                            print('=============================================================')
                            repeat_num = repeat_num+1
                            continue
                        else :
                               break
                    last_height = str(driver.execute_script('return document.documentElement.scrollHeight'))
                
                if repeat_num>20:
                    lst_false.append(user_id)
                    print(user_id, ": 반복 횟수가 20번을 넘어 다음으로 넘어감(별도 저장됨)")
                    continue
                else:
                    rated_mvs = driver.find_elements_by_class_name('e3fgkal0')
                    for rated_mv in rated_mvs:
                        mv_code = rated_mv.find_element_by_tag_name('a').get_attribute('href').split('/')[-1]
                        ratings = float(rated_mv.find_element_by_class_name('e3fgkal4').text.lstrip('★'))
                        lst_true.append((user_id, mv_code, ratings))
                        temp_.append((user_id, mv_code, ratings))
            print('user_id : ', user_id, '평가한 영화 개수 : ', len(temp_))        
        kwd_true = ['user_id', 'mv_code', 'ratings']
        cont_true = list(zip(*lst_true))
        df_true = DataFrame(dict(zip(kwd_true, cont_true)))
        if lst_false:
            kwd_false = 'user_id'
            cont_false = lst_false
            df_false = DataFrame({kwd_false:cont_false})
        else:
            df_false = DataFrame()
    finally:
        driver.quit()
    return(df_true, df_false)

In [16]:
options = webdriver.ChromeOptions()

prefs = {
    'profile.default_content_setting_values': {
          'cookies': 2, 'images': 2, 'plugins': 2
        , 'popups': 2, 'geolocation': 2, 'notifications': 2
        , 'auto_select_certificate': 2, 'fullscreen': 2, 'mouselock': 2
        , 'mixed_script': 2, 'media_stream': 2, 'media_stream_mic': 2
        , 'media_stream_camera': 2, 'protocol_handlers': 2, 'ppapi_broker': 2
        , 'automatic_downloads': 2, 'midi_sysex': 2, 'push_messaging': 2
        , 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 'protected_media_identifier': 2
        , 'app_banner': 2, 'site_engagement': 2, 'durable_storage': 2
    }
}

options.add_experimental_option('prefs', prefs)
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('headless')
options.add_argument("disable-gpu")
# console창에서 계속 에러가 발생하고 있는 것을 발견
# [0423/180816.169:INFO:CONSOLE(0)] "Uncaught (in promise) TypeError: Failed to fetch", source: https://watcha.com/sw.js (0)
# same origin policy는 결국 클라이언트인 웹 브라우저가 요청을 해도 되는지 판단해서 결정하는 것으로,
# 이 과정만 무시한다면 어디든 요청을 못할 이유는 없다
# 크롬같은 웹 브라우저들은 실행시 커맨드라인 옵션을 통해서 외부 도메인 요청가능 여부를 확인하는 동작을 무시하는 것이 가능.
# 동일출처정책(same-origin-policy) 참고
# https://enterkey.tistory.com/409
options.add_argument("--disable-web-security")

In [17]:
def run_get_user_data(start_num, end_num, thread_num):    
    split_num = int((end_num-start_num)/thread_num)
    q = Queue(maxsize=thread_num)
    for _ in range(q.maxsize):
        driver = webdriver.Chrome('./driver/chromedriver.exe'
                                 , options = options
                                 )
        q.put(driver)
        
    with ThreadPool(thread_num) as tp:
        user_data = tp.starmap(
            user_info
            , [
                (login(q.get()), df[start_num+i*split_num:start_num+(i+1)*split_num])
                                  for i in range(0,thread_num)
            ]
        )
    pre_df_lst = list(zip(*user_data))
    df_true = pd.concat(pre_df_lst[0])
    df_false = pd.concat(pre_df_lst[1])
    return (df_true, df_false)

In [18]:
df

,id,rating_count
0,17ovVgGBL5zyn,20120.0
1,YMKqmjXMkvloD,16657.0
2,Qgy51zEAZxDjk,11152.0
3,XZBm5RYOavd46,10716.0
4,WRQxDOw1V5dl9,10676.0
...,...,...
207788,zNM5NbVn2526j,6.0
207789,87Gv7GXjJvE6o,6.0
207790,WRQxDPYZnxdl9,6.0
207791,3BnvwnbbDKqPA,6.0


In [19]:
df[5000:10000]

,id,rating_count
5000,yKZx3wk4Gx4dJ,1473.0
5001,DgwxAP6Am5rMj,1473.0
5002,6ADvGpZPd5zZl,1472.0
5003,4WLxZddomvroA,1472.0
5004,8nPvy1kmDvYo0,1472.0
...,...,...
9995,yKZx3rl0Z54dJ,1199.0
9996,7gdxd39yy5GYJ,1199.0
9997,JgkqlAzWavX0b,1199.0
9998,nb4xk9Yj3vOAz,1198.0


In [ ]:
# ======================
start_num = 5000
end_num = 10000
    
jump = 200 # 컴퓨터를 빨리 종료해야되는 경우에만 수를 적게 입력(driver가 자주 켜지면서 속도 감소하기 때문)
thread_num = 5
# ======================
for i in tqdm_notebook(range(int((end_num-start_num)/jump))):
    user_ratings = run_get_user_data(start_num + i*jump , start_num + (i+1)*jump, thread_num)
    if user_ratings[1].empty:
        user_ratings[0].to_csv(f'./watcha_user/temp2/user_data{start_num + i*jump}-{start_num + (i+1)*jump}.csv')
    else:
        user_ratings[0].to_csv(f'./watcha_user/temp2/user_data{start_num + i*jump}-{start_num + (i+1)*jump}.csv')
        user_ratings[1].to_csv(f'./watcha_user/temp3/error_user_data{start_num + i*jump}-{start_num + (i+1)*jump}.csv')

user_id :  DgwxAjmEgqrMj 평가한 영화 개수 :  1460
user_id :  djaxbr9mkxLw8 평가한 영화 개수 :  1464
user_id :  j4PxOYAdovp0Q 평가한 영화 개수 :  1465
user_id :  yKZx3wk4Gx4dJ 평가한 영화 개수 :  1473
user_id :  Mr95nygQ7vZPG 평가한 영화 개수 :  1469
user_id :  DgwxAP6Am5rMj 평가한 영화 개수 :  1473
user_id :  6ewxaK12QvQ1m 평가한 영화 개수 :  1463
user_id :  a9L5PVkE0qwg8 평가한 영화 개수 :  1460
user_id :  Mr95nPgmNqZPG 평가한 영화 개수 :  1466
user_id :  6nkPvr8yoxgar 평가한 영화 개수 :  1469
user_id :  dP8v610XQqWeJ 평가한 영화 개수 :  1463
user_id :  6ADvGpZPd5zZl 평가한 영화 개수 :  1472
user_id :  6ewxaBkVZqQ1m 평가한 영화 개수 :  1460
user_id :  2mwvg2kArqMa7 평가한 영화 개수 :  1466
user_id :  R6OvKBwJAqN8V 평가한 영화 개수 :  1469
user_id :  JgAx8j1aDvLbO 평가한 영화 개수 :  1463
user_id :  4WLxZddomvroA 평가한 영화 개수 :  1472
user_id :  Mr95nokNBxZPG 평가한 영화 개수 :  1460
user_id :  Q9L5pG2BbvNb0 평가한 영화 개수 :  1466
user_id :  K6ExjbByzxX1O 평가한 영화 개수 :  1468
user_id :  jae5Wa8Ppq1P3 평가한 영화 개수 :  1463
user_id :  8nPvy1kmDvYo0 평가한 영화 개수 :  1472
user_id :  Pdjaxb1WE5Lw8 평가한 영화 개수 :  1460
user_id :  

user_id :  ld0q042jYq6Xn 평가한 영화 개수 :  1459
user_id :  OkexJYmJe5dbw 평가한 영화 개수 :  1457
user_id :  yKZx3QNYDv4dJ 평가한 영화 개수 :  1470
user_id :  6Qgy51yrD5Djk 평가한 영화 개수 :  1463
user_id :  Q9L5p1rQQxNb0 평가한 영화 개수 :  1466
user_id :  dP8v6k2eM5WeJ 평가한 영화 개수 :  1460

user_id :  YaR5YMk7E5gX1 평가한 영화 개수 :  1457

user_id :  zNM5Nb1n2526j 평가한 영화 개수 :  1470

user_id :  GRE9523odxQ72 평가한 영화 개수 :  1463

user_id :  g64qzJAEMqER0 평가한 영화 개수 :  1466



user_id :  YaR5Y6LryxgX1 평가한 영화 개수 :  1453
user_id :  yKZx3V4k254dJ 평가한 영화 개수 :  1451
user_id :  nkPvrP8wjxgar 평가한 영화 개수 :  1442
user_id :  6ADvGbroLqzZl 평가한 영화 개수 :  1457
user_id :  ld0q0yk4Gv6Xn 평가한 영화 개수 :  1446
user_id :  OkexJRpnmxdbw 평가한 영화 개수 :  1450
user_id :  ZWpqMz471vrkn 평가한 영화 개수 :  1456
user_id :  yKZx31Vjov4dJ 평가한 영화 개수 :  1442
user_id :  Qgy513p6MqDjk 평가한 영화 개수 :  1457
user_id :  WwRqoPnYyvlzB 평가한 영화 개수 :  1446
user_id :  3Bnvw7p29vPAY 평가한 영화 개수 :  1442
user_id :  3BnvwgLXKvPAY 평가한 영화 개수 :  1450
user_id :  YMKqmQ2jlqloD 평가한 영화 개수 :  1454
g64qz20MRqER0 : 개수가 맞지않아 1번째 재실행 중(오차 -1450개)
user_id :  WRQxDwlwV5dl9 평가한 영화 개수 :  1446
user_id :  g64qzDP2R5ER0 평가한 영화 개수 :  1461
g64qz20MRqER0 : 개수가 맞지않아 2번째 재실행 중(오차 -1450개)
g64qz20MRqER0 : 개수가 맞지않아 3번째 재실행 중(오차 -1450개)
g64qz20MRqER0 : 개수가 맞지않아 4번째 재실행 중(오차 -1450개)
g64qz20MRqER0 : 개수가 맞지않아 5번째 재실행 중(오차 -1450개)
g64qz20MRqER0 : 개수가 맞지않아 6번째 재실행 중(오차 -1450개)
g64qz20MRqER0 : 개수가 맞지않아 7번째 재실행 중(오차 -1450개)
g64qz20MRqER0 : 개수가 맞지않아 8번째 재실행 

user_id :  4WLxZ8dakvroA 평가한 영화 개수 :  1441
user_id :  7gdxdkkk25GYJ 평가한 영화 개수 :  1444
user_id :  Pznx9wKjWv761 평가한 영화 개수 :  1457
user_id :  WRQxD06Mrxdl9 평가한 영화 개수 :  1452
user_id :  2mwvgekK4xMa7 평가한 영화 개수 :  1447
user_id :  NP9vLL6dZv6kl 평가한 영화 개수 :  1441
user_id :  nb4xkOmmGxOAz 평가한 영화 개수 :  1443
user_id :  ZWpqM6zDkvrkn 평가한 영화 개수 :  1456
user_id :  djaxbPM97vLw8 평가한 영화 개수 :  1453
user_id :  6ewxaobVpvQ1m 평가한 영화 개수 :  1447
user_id :  djaxbJbrwxLw8 평가한 영화 개수 :  1441
user_id :  nkPvrJd4Ovgar 평가한 영화 개수 :  1443
user_id :  6ADvGJ4QWqzZl 평가한 영화 개수 :  1455
user_id :  l17ovVN0Bvzyn 평가한 영화 개수 :  1452
user_id :  g64qzJ8deqER0 평가한 영화 개수 :  1447
user_id :  36lvXw6bPqXdn 평가한 영화 개수 :  1441
user_id :  2mwvgzBPwqMa7 평가한 영화 개수 :  1445
user_id :  nkPvrl3Qmxgar 평가한 영화 개수 :  1456
user_id :  36lvXy36o5Xdn 평가한 영화 개수 :  1447
user_id :  djaxbJpRMxLw8 평가한 영화 개수 :  1452
user_id :  YMKqmPN7lxloD 평가한 영화 개수 :  1441
user_id :  Qgy51WbnXqDjk 평가한 영화 개수 :  1443
user_id :  QJgAx8G1V5LbO 평가한 영화 개수 :  1452
user_id :  

user_id :  RE952mV4EvQ72 평가한 영화 개수 :  1430
user_id :  YMKqmp767xloD 평가한 영화 개수 :  1439
user_id :  OkexJ4BB4vdbw 평가한 영화 개수 :  1433
user_id :  R6OvKVJW6qN8V 평가한 영화 개수 :  1437
user_id :  yKZx3YweGv4dJ 평가한 영화 개수 :  1428
user_id :  jae5WrEyav1P3 평가한 영화 개수 :  1430
user_id :  DgwxA2rAdqrMj 평가한 영화 개수 :  1439
user_id :  7gdxdmmEXqGYJ 평가한 영화 개수 :  1433
user_id :  wR6OvKJX6qN8V 평가한 영화 개수 :  1437
user_id :  OkexJ4Brrvdbw 평가한 영화 개수 :  1427
user_id :  WwRqo98M15lzB 평가한 영화 개수 :  1430
user_id :  6NW5QG3lZq1Yo 평가한 영화 개수 :  1434
user_id :  nkPvr64rovgar 평가한 영화 개수 :  1433
user_id :  8nPvyGLmoxYo0 평가한 영화 개수 :  1428
user_id :  VRZv4jLmnxr6y 평가한 영화 개수 :  1435
user_id :  g64qzBJzeqER0 평가한 영화 개수 :  1430
user_id :  g64qzZdeLqER0 평가한 영화 개수 :  1433
user_id :  VRZv4n71Bxr6y 평가한 영화 개수 :  1439
user_id :  ZWpqMB3zXvrkn 평가한 영화 개수 :  1428
user_id :  Mr95nNnjb5ZPG 평가한 영화 개수 :  1435
user_id :  zNM5NYRz7v26j 평가한 영화 개수 :  1430
user_id :  ZBm5R0l69qd46 평가한 영화 개수 :  1433
user_id :  Mr95nWPpBxZPG 평가한 영화 개수 :  1439
user_id :  

...

user_id :  WwRqoLDn1vlzB 평가한 영화 개수 :  1341
user_id :  R6OvKMBlLxN8V 평가한 영화 개수 :  1338
user_id :  K6ExjapYw5X1O 평가한 영화 개수 :  1336
user_id :  JgkqlgJV0xX0b 평가한 영화 개수 :  1334
user_id :  ZWpqMeJ7kqrkn 평가한 영화 개수 :  1343
user_id :  dP8v6NWEavWeJ 평가한 영화 개수 :  1338
user_id :  nkPvrZzzDvgar 평가한 영화 개수 :  1343
user_id :  DgwxA2W8bqrMj 평가한 영화 개수 :  1341
user_id :  WwRqo7JWyqlzB 평가한 영화 개수 :  1336
user_id :  K6Exj2lgwxX1O 평가한 영화 개수 :  1334
user_id :  YMKqm60XEvloD 평가한 영화 개수 :  1338
user_id :  3Bnvw10z0xPAY 평가한 영화 개수 :  1341
user_id :  JgkqlKzGmxX0b 평가한 영화 개수 :  1343
user_id :  87Gv7pkpG5E6o 평가한 영화 개수 :  1336
user_id :  JZ4vBpR0ZxRXO 평가한 영화 개수 :  1334
user_id :  D4WLxZAr5roAn 평가한 영화 개수 :  1342
user_id :  Qgy51YgOD5Djk 평가한 영화 개수 :  1342
user_id :  24BqEzrQNvrjn 평가한 영화 개수 :  1343
user_id :  Q9L5pL2wb5Nb0 평가한 영화 개수 :  1336
user_id :  17ovVjXQlvzyn 평가한 영화 개수 :  1334
user_id :  6ADvGD1BA5zZl 평가한 영화 개수 :  1337
user_id :  17ovV7J215zyn 평가한 영화 개수 :  1344
user_id :  87Gv7N2dVqE6o 평가한 영화 개수 :  1347
user_id :  

user_id :  3BnvwwPJ9vPAY 평가한 영화 개수 :  1335
user_id :  Pznx9BX8aq761 평가한 영화 개수 :  1341
user_id :  YMKqm6eXlvloD 평가한 영화 개수 :  1332
user_id :  WwRqod7O0vlzB 평가한 영화 개수 :  1336
user_id :  8BRveYJ6lv6VJ 평가한 영화 개수 :  1340
user_id :  DgwxAD3ZE5rMj 평가한 영화 개수 :  1334

user_id :  Mr95n0BzjqZPG 평가한 영화 개수 :  1341

user_id :  24BqE2YpNvrjn 평가한 영화 개수 :  1332

user_id :  ZWpqMnW2Vvrkn 평가한 영화 개수 :  1336

user_id :  pdP8v6PBkxWeJ 평가한 영화 개수 :  1338



user_id :  K6Exj8wZ1vX1O 평가한 영화 개수 :  1330
user_id :  YMKqmnp17vloD 평가한 영화 개수 :  1327
user_id :  Qgy51GG2ZqDjk 평가한 영화 개수 :  1332
user_id :  YaR5YE3a95gX1 평가한 영화 개수 :  1325
user_id :  JZ4vB6wyzvRXO 평가한 영화 개수 :  1323
user_id :  ZWpqMY4mDqrkn 평가한 영화 개수 :  1330
user_id :  2mwvgPNAO5Ma7 평가한 영화 개수 :  1332
user_id :  NP9vL9Mn856kl 평가한 영화 개수 :  1327
user_id :  7gdxdkj8L5GYJ 평가한 영화 개수 :  1325
user_id :  YaR5YeXko5gX1 평가한 영화 개수 :  1323
user_id :  JZ4vBbQW4vRXO 평가한 영화 개수 :  1330
user_id :  J3Bnvwo7BvPAY 평가한 영화 개수 :  1325
user_id :  7gdxdWBPR5GYJ 평가한 영화 개수 :  1332
user_id :  7gdxdW0j95GYJ 평가한 영화 개수 :  1327
user_id :  g64qzZdPgqER0 평가한 영화 개수 :  1323
user_id :  WRQxDM340qdl9 평가한 영화 개수 :  1330
user_id :  6NW5QLK3wv1Yo 평가한 영화 개수 :  1323
user_id :  8BRve3DlQx6VJ 평가한 영화 개수 :  1325
user_id :  6ADvGQppmxzZl 평가한 영화 개수 :  1327
user_id :  Qgy51L12BvDjk 평가한 영화 개수 :  1335
user_id :  Qgy51G0LXqDjk 평가한 영화 개수 :  1330
user_id :  36lvXb2MPxXdn 평가한 영화 개수 :  1327
user_id :  zNM5NYEolv26j 평가한 영화 개수 :  1323
user_id :  

user_id :  NP9vLELVo56kl 평가한 영화 개수 :  1329
user_id :  24BqEjaBOvrjn 평가한 영화 개수 :  1323
user_id :  24BqEAPe1qrjn 평가한 영화 개수 :  1321
user_id :  24BqEJz34vrjn 평가한 영화 개수 :  1326
user_id :  4WLxZjA86xroA 평가한 영화 개수 :  1340
user_id :  4WLxZpmW8vroA 평가한 영화 개수 :  1328

user_id :  WwRqoYQLX5lzB 평가한 영화 개수 :  1323

user_id :  ZWpqMPg9Nxrkn 평가한 영화 개수 :  1320

user_id :  JgAx88YapxLbO 평가한 영화 개수 :  1330

user_id :  yKZx3gAlRx4dJ 평가한 영화 개수 :  1325



user_id :  djaxbJ9ojxLw8 평가한 영화 개수 :  1319
user_id :  g64qz1BlRxER0 평가한 영화 개수 :  1313
user_id :  g64qzbE40qER0 평가한 영화 개수 :  1314
user_id :  67gdxdPN9xGYJ 평가한 영화 개수 :  1317
user_id :  NP9vL2ZN9x6kl 평가한 영화 개수 :  1322
user_id :  OkexJ1VoK5dbw 평가한 영화 개수 :  1318
user_id :  8nPvyz4Vo5Yo0 평가한 영화 개수 :  1319
user_id :  g64qzQAogxER0 평가한 영화 개수 :  1313
user_id :  4WLxZRX2ExroA 평가한 영화 개수 :  1316
user_id :  K6ExjO9a8qX1O 평가한 영화 개수 :  1317
user_id :  Qgy51KGGeqDjk 평가한 영화 개수 :  1321
user_id :  6ADvGLNbA5zZl 평가한 영화 개수 :  1311
user_id :  r2mwvgOM4vMa7 평가한 영화 개수 :  1319
user_id :  WwRqobQzDxlzB 평가한 영화 개수 :  1314
user_id :  nb4xk13N3vOAz 평가한 영화 개수 :  1316
